In [9]:
import ee
# Initialize ee and authenticate 
#ee.Authenticate()
ee.Initialize()

In [10]:
import math
import geemap
import numpy as np
import pandas as pd
import os

In [11]:
## Bring in buffered range map as 'region' 
region = ee.FeatureCollection('projects/GEE_CSP/pf-bobwhite/bobwhite_model_states')
geometry = ee.Feature(ee.FeatureCollection(region).first())
conus_geom = ee.FeatureCollection("projects/GEE_CSP/thirty-by-thirty/aoi_conus")
conus_img = ee.Image("projects/GEE_CSP/thirty-by-thirty/aoi_conus_mask")

# export scale and projection
# scale = 250
projection = ee.Projection('EPSG:5070') # stand-in for now. Figure out best projection to use 

In [12]:
# Focal mean
def focal_mean(image, radius, unit, name):
    names = image.bandNames().getInfo()
    new_names = [s + name for s in names]
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.mean()).rename(new_names)

# Focal median
def focal_median(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.median())
    
# Focal SD
def focal_sd(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.stdDev())

# Define a function to calculate the absolute deviation of values from the mean within the window - window size is in meters 
def surface_sa(image, window_size, name):
    names = image.bandNames().getInfo()
    new_names = [s + name for s in names]
    kernel = ee.Kernel.circle(radius=window_size, units='meters')
    mean = image.reduceNeighborhood(ee.Reducer.mean(), kernel)
    deviation = image.subtract(mean)
    absolute_deviation = deviation.abs()
    return absolute_deviation.reduceNeighborhood(ee.Reducer.mean(), kernel).rename(new_names)


In [14]:
geometry = ee.Feature(ee.FeatureCollection("projects/GEE_CSP/pf-bobwhite/bobwhite_model_states").first());
##---------- Define the years that you want to export --------------
##---------- End year is inclusive in this case  ------------------
yearStart = 2016
yearEnd = 2021

## -------------- Define the plant functional types (PFTs) that you want to export --------------
## PFTs are "AFGC" (Annual forb and grass cover), "BG" (bare ground), "LTR" (litter), 
## "PFGC" (perennial forb and grass cover), "SHR" (shrub cover), and "TREE" (tree cover)
## Select Annual forb and grass cover, perennial forb and grass cover, shrub cover, and tree cover 
PFTs = ee.List(['AFG', 'PFG', 'SHR', 'TRE', 'BGR']);

cover = ee.ImageCollection("projects/rangeland-analysis-platform/vegetation-cover-v3")
## ------------- Select the PFTs for processing as defined by User  --------------
cover_toExport = cover.select(PFTs)


### Filter RAP cover ImageCollection into yearly sets
rap_cover_2016 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2016]))).toBands()
rap_cover_2017 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2017]))).toBands()
rap_cover_2018 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2018]))).toBands()
rap_cover_2019 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2019]))).toBands()
rap_cover_2021 = cover_toExport.filter(ee.Filter.inList('year', ee.List([2021]))).toBands()

### Combine all into single multiband image 
rap_cover_all = ee.Image([rap_cover_2016, rap_cover_2017, rap_cover_2018, rap_cover_2019, rap_cover_2021])

rap_cover_all_band_names = rap_cover_all.bandNames()
print('rap_cover_2019 Band names:', rap_cover_all_band_names.getInfo())  # ee.List of band names


rap_cover_2019 Band names: ['2016_AFG', '2016_PFG', '2016_SHR', '2016_TRE', '2016_BGR', '2017_AFG', '2017_PFG', '2017_SHR', '2017_TRE', '2017_BGR', '2018_AFG', '2018_PFG', '2018_SHR', '2018_TRE', '2018_BGR', '2019_AFG', '2019_PFG', '2019_SHR', '2019_TRE', '2019_BGR', '2021_AFG', '2021_PFG', '2021_SHR', '2021_TRE', '2021_BGR']


In [21]:
# Map the function over the image collection
sa_img = surface_sa(rap_cover_all, 5000, "_5km_sa").clip(geometry)

sa_img_band_names = sa_img.bandNames()
print('sa_img_band_names:', sa_img_band_names.getInfo())  # ee.List of band names

sa_img_band_names: ['2016_AFG_5km_sa', '2016_PFG_5km_sa', '2016_SHR_5km_sa', '2016_TRE_5km_sa', '2016_BGR_5km_sa', '2017_AFG_5km_sa', '2017_PFG_5km_sa', '2017_SHR_5km_sa', '2017_TRE_5km_sa', '2017_BGR_5km_sa', '2018_AFG_5km_sa', '2018_PFG_5km_sa', '2018_SHR_5km_sa', '2018_TRE_5km_sa', '2018_BGR_5km_sa', '2019_AFG_5km_sa', '2019_PFG_5km_sa', '2019_SHR_5km_sa', '2019_TRE_5km_sa', '2019_BGR_5km_sa', '2021_AFG_5km_sa', '2021_PFG_5km_sa', '2021_SHR_5km_sa', '2021_TRE_5km_sa', '2021_BGR_5km_sa']


In [22]:
vis_params = {
    'bands': '2019_AFG_5km_sa',
    'min': 0,
    'max': 50,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
## Plot to check 
Map = geemap.Map(center=(40, -100), zoom=4)
Map.addLayer(sa_img, vis_params, "Tree SA", True)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

Decompressing WhiteboxTools_darwin_amd64.zip ...
WhiteboxTools package directory: /Users/patrickfreeman-csp/opt/miniconda3/lib/python3.9/site-packages/whitebox
